## Week 3 - Deep Learning Computation

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import tensorflow as tf

### 5.1. Layers and Blocks

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0487, -0.0525, -0.0506, -0.0665,  0.0252, -0.0694, -0.1188,  0.1032,
          0.1927, -0.0783],
        [-0.0877, -0.1212, -0.0357,  0.0351,  0.0981, -0.0303, -0.1346,  0.0302,
          0.1725, -0.0341]], grad_fn=<AddmmBackward0>)

#### 5.1.1. A Custom Block

In [3]:
class MLP(nn.Module):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def forward(self, X):
        # Note here we use the funtional version of ReLU defined in the
        # nn.functional module.
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.3339,  0.1305,  0.0954, -0.2233, -0.0152,  0.0392,  0.2784, -0.1912,
         -0.1052,  0.1552],
        [-0.1597,  0.1051,  0.1728, -0.1450, -0.1045,  0.0066,  0.2006, -0.1645,
         -0.1363,  0.1957]], grad_fn=<AddmmBackward0>)

#### 5.1.2. The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1926, -0.1782,  0.1521, -0.0555,  0.1252,  0.3248,  0.2715,  0.0362,
         -0.0037,  0.0032],
        [-0.2325, -0.1098,  0.1083, -0.0710,  0.1180,  0.0394,  0.2322,  0.0769,
         -0.1499,  0.1010]], grad_fn=<AddmmBackward0>)

#### 5.1.3. Executing Code in the Forward Propagation Function¶

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.1438, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3862, grad_fn=<SumBackward0>)

### 5.2. Parameter Management

In [10]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.3931],
        [0.3945]], grad_fn=<AddmmBackward0>)

#### 5.2.1. Parameter Access

In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0104,  0.1644,  0.0760,  0.1985,  0.1540,  0.2112,  0.0426, -0.2754]])), ('bias', tensor([0.3273]))])


##### 5.2.1.1. Targeted Parameters

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3273], requires_grad=True)
tensor([0.3273])


In [13]:
net[2].weight.grad == None

True

##### 5.2.1.2. All Parameters at Once

In [14]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
net.state_dict()['2.bias'].data

tensor([0.3273])

##### 5.2.1.3. Collecting Parameters from Nested Blocks

In [16]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1585],
        [-0.1585]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
rgnet[0][1][0].bias.data

tensor([ 0.1254, -0.3874, -0.3726, -0.0309, -0.4700, -0.4400,  0.1512, -0.0950])

#### 5.2.2. Parameter Initialization

##### 5.2.2.1. Built-in Initialization

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0020, -0.0036, -0.0091,  0.0156]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5925, -0.6690,  0.5378, -0.4331])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


##### 5.2.2.2. Custom Initialization

In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -5.8142, -0.0000,  0.0000],
        [-0.0000, -0.0000, -9.7008, -6.4985]], grad_fn=<SliceBackward0>)

In [23]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -4.8142,  1.0000,  1.0000])

#### 5.2.3. Tied Parameters

In [24]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 5.3. Deferred Initialization

#### 5.3.1. Instantiating a Network

In [25]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [26]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [27]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

### 5.4. Custom Layers

#### 5.4.1. Layers without Parameters

In [28]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [29]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [30]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [31]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-4.0745e-09, grad_fn=<MeanBackward0>)

#### 5.4.2. Layers with Parameters

In [32]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [33]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.5442,  0.3826, -0.0493],
        [ 0.2097, -0.4264,  0.1550],
        [-1.1745, -0.1134, -0.0731],
        [-0.5968,  0.9674, -3.0821],
        [-1.1078,  1.3804,  1.2691]], requires_grad=True)

In [34]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.2448, 0.0000],
        [0.0000, 1.5337, 0.0000]])

In [35]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.4253],
        [0.0000]])

### 5.5. File I/O

#### 5.5.1. Loading and Saving Tensors

In [36]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [37]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [38]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [39]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

#### 5.5.2. Loading and Saving Model Parameters

In [40]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [41]:
torch.save(net.state_dict(), 'mlp.params')

In [42]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [43]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 5.6. GPUs

In [44]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



#### 5.6.1. Computing Devices

In [45]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [46]:
torch.cuda.device_count()

0

In [47]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

#### 5.6.2. Tensors and GPUs

In [48]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

##### 5.6.2.1. Storage on the GPU

In [49]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [50]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.7320, 0.6156, 0.3022],
        [0.4793, 0.0179, 0.9257]])

##### 5.6.2.2. Copying

In [51]:
Z = X #Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [52]:
Y + Z

tensor([[1.7320, 1.6156, 1.3022],
        [1.4793, 1.0179, 1.9257]])

In [53]:
#Z.cuda(1) is Z

#### 5.6.3. Neural Networks and GPUs

In [54]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [55]:
net(X)

tensor([[1.0338],
        [1.0338]], grad_fn=<AddmmBackward0>)

In [56]:
net[0].weight.data.device

device(type='cpu')